In [5]:
import glob
import base64
import os.path
import numpy as np
import pandas as pd

from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow

from email import encoders
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

import access_gmail as ag

In [6]:
from importlib import reload
reload(ag)

<module 'access_gmail' from '/Users/darthpatel/Library/CloudStorage/Dropbox/FunProjects/Gmail-API/access_gmail.py'>

## Scopes check

In [7]:
SCOPES = ag.SCOPES
SCOPES

['https://www.googleapis.com/auth/spreadsheets',
 'https://www.googleapis.com/auth/spreadsheets.readonly',
 'https://www.googleapis.com/auth/drive',
 'https://www.googleapis.com/auth/gmail.send',
 'https://www.googleapis.com/auth/drive.file',
 'https://www.googleapis.com/auth/gmail.labels',
 'https://www.googleapis.com/auth/gmail.compose',
 'https://www.googleapis.com/auth/gmail.modify',
 'https://www.googleapis.com/auth/drive.activity',
 'https://mail.google.com/',
 'https://www.googleapis.com/auth/contacts.readonly',
 'https://www.googleapis.com/auth/drive.metadata.readonly',
 'https://www.googleapis.com/auth/gmail.addons.current.action.compose',
 'https://www.googleapis.com/auth/gmail.addons.current.message.action']

## Credentials generation or load

In [8]:
creds = ag.create_token()
# Initialize the Gmail API
service = build('gmail', 'v1', credentials=creds)
people_service = build('people', 'v1', credentials=creds)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=5236653841-gn2vg6si5ld9u9c601utukk1vnan3ulk.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A58165%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fspreadsheets+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fspreadsheets.readonly+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.send+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.file+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.labels+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.compose+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.modify+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.activity+https%3A%2F%2Fmail.google.com%2F+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcontacts.readonly+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.metadata.readonly+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.addons.current.action.compose+https%3A%2F%2Fwww.g

## Email labels and list of emails

In [5]:


# Define the label you want to filter by (replace 'YOUR_LABEL_ID' with your actual label ID)
label_id = 'AoT Subscribed'

# Get emails with the specified label

# emails = ag.get_emails_by_label(service, label_id)
############## - get emails by label start
results = service.users().messages().list(userId='me', labelIds=[label_id]).execute()
messages = results.get('messages', [])
emails = []
for message in messages:
    msg = service.users().messages().get(userID='me',id=message['id']).execute()
    email_data = {
        'id': msg['id'],
        'snippet': msg['snippet'],
        'payload': msg['payload']
    }
    emails.append(email_data)



##############

print(f'Found {len(emails)} emails with label {label_id}')
for email in emails:
    print(f'Email ID: {email["id"]}, Snippet: {email["snippet"]}')

HttpError: <HttpError 400 when requesting https://gmail.googleapis.com/gmail/v1/users/me/messages?labelIds=AoT+Subscribed&alt=json returned "Invalid label: AoT Subscribed". Details: "[{'message': 'Invalid label: AoT Subscribed', 'domain': 'global', 'reason': 'invalidArgument'}]">

In [ ]:
contacts = ag.get_contacts(people_service)

In [ ]:
contacts

## HTML content

In [52]:
event_day = 'THURS.'
event_month = 'June'
event_date = '27th'
event_edition = 'Summer Solstice Edition'
location = 'Jackalope Brewing – Tap Room'
speaker1_name = 'Dr. Chayan Chatterjee'
speaker1_title = 'Listening to the Cosmic Orchestra Using A.I.'
speaker2_name = 'Dr. Aaron Stemo'
speaker2_title = 'Black Holes Don\'t Suck'
eventbrite_link = 'https://www.eventbrite.com'
other_news = ''  # additional messages

html_content2 = f"""
<!DOCTYPE html>
<html>
<head>
    <title>Astronomy on Tap - {event_edition}</title>
</head>
<body>
    <p>Dear AoT-ers!</p>
    <p>We are pleased to announce that our <strong>{event_edition}</strong> event for
    Astronomy on Tap will be held on <strong>{event_day} {event_month} {event_date}</strong> at the <strong>{location}</strong>!</p>
    <p>This event will feature 2 amazing speakers:</p>
    <ul>
        <li><strong>{speaker1_name}</strong><br><em>{speaker1_title}</em></li>
        <li><strong>{speaker2_name}</strong><br><em>{speaker2_title}</em></li>
    </ul>
    <p>There will be trivia and more prizes after! So come on out after a long day of hard work in the middle of the week and relax while we bombard you with fun astro facts, beer, and prizes!</p>
    <p>Make sure to <a href="{eventbrite_link}" target="_blank">RSVP for the free event at EventBrite</a>.</p>
    <p>Please share to all and any who might be interested!</p>
    <p><strong>Doors 7pm | Show 7:30 pm</strong></p>
    <p><a href="https://www.google.com/maps/place/Jackalope+Brewing" target="_blank">Parking Instructions</a></p>
    <p>{other_news}</p>
    <p>If you wish to unsubscribe, reply to this email</p>
    <p>Sincerely,
    <br>AoT Nashville Team</p>
<br>
<div>
    <div style="text-align:center; color:rgb(34,34,34)">
        <img data-aii="CiExR3hhR1hFMERrVm9idFl0QWRzUWZ3SGhSemtIU0dGZ0I" width="96" height="96" 
             src="https://ci3.googleusercontent.com/mail-sig/AIorK4wR7MgccJn6Wi3LFcw0z4QsBsos9nAKfBYh9BxpNHb006gIZIZwnz_hNMZQFongZ6f5_uEHKaQ" 
             data-os="https://lh3.googleusercontent.com/d/1GxaGXE0DkVobtYtAdsQfwHhRzkHSGFgB">
        <br>
    </div>
    <div style="text-align:center; color:rgb(34,34,34)">
        Astronomy on Tap - Nashville
    </div>
    <div style="text-align:center; color:rgb(34,34,34)">
        <b>
            <a href="https://www.instagram.com/aotnashville/" style="color:rgb(17,85,204)" target="_blank">Instagram</a>&nbsp;|&nbsp;
            <a href="https://www.facebook.com/aotnashville" target="_blank">Facebook</a>
        </b>
    </div>
</div>

</body>
</html>
"""

In [6]:
sender = 'aotnahsville@gmail.com'
subject = '[Astro on Tap] -- blah blah'
message_text = html_content

In [ ]:
message = MIMEMultipart()
message['to'] = 'anakha.1@gmail.com'
message['from'] = sender
message['subject'] = subject
msg = MIMEText(message_text)
message.attach(msg)
raw = base64.urlsafe_b64encode(message.as_bytes()).decode()

In [ ]:
message_raw = {'raw':raw}
ag.send_message(service,'me',message_raw)

In [11]:
message = EmailMessage()

message.set_content("This is automated draft mail")

message["To"] = "anakha.1@gmail.com"
message["From"] = sender
message["Subject"] = "Automated draft"

# encoded message
encoded_message = base64.urlsafe_b64encode(message.as_bytes()).decode()

create_message = {"message": {"raw": encoded_message}}
# pylint: disable=E1101
draft = (
    service.users()
    .drafts()
    .create(userId="me", body=create_message)
    .execute()
)


In [13]:
print(f'Draft id: {draft["id"]}\nDraft message: {draft["message"]}')


Draft id: r5379049820878808303
Draft message: {'id': '19026c70d684a3be', 'threadId': '19026c70d684a3be', 'labelIds': ['DRAFT']}


In [14]:
draft['message']

{'id': '19026c70d684a3be',
 'threadId': '19026c70d684a3be',
 'labelIds': ['DRAFT']}

In [21]:
message = MIMEMultipart('alternative') #EmailMessage()
#message.set_content(html_content)
message['To'] = 'ripatel272@gmail.com,anakha.1@gmail.com,malvikag12@gmail.com'
message['From'] = sender
message['Subject'] = 'Test'

msg = MIMEText(html_content,'html')
message.attach(msg)
encoded_message = base64.urlsafe_b64encode(message.as_bytes()).decode()
create_message = {'raw':encoded_message}
send_message = service.users().messages().send(userId="me", body=create_message).execute()

## add attachment google drive local to email message

In [57]:
import glob
file_path = '/Users/darthpatel/Library/CloudStorage/GoogleDrive-aotnashville@gmail.com/My Drive/Event_flyers/'
# file_path = '/Users/darthpatel/Downloads/'
files_2_attach = glob.glob(file_path + '*June*')
files_2_attach

['/Users/darthpatel/Library/CloudStorage/GoogleDrive-aotnashville@gmail.com/My Drive/Event_flyers/AoT_June2024.pdf',
 '/Users/darthpatel/Library/CloudStorage/GoogleDrive-aotnashville@gmail.com/My Drive/Event_flyers/AoT_June2024_Insta.png']

In [58]:
message = MIMEMultipart('resumable') #EmailMessage()
#message.set_content(html_content)
message['To'] = 'ripatel272@gmail.com,anakha.1@gmail.com'#,malvikag12@gmail.com'
message['From'] = sender
message['Subject'] = 'Test'
message['bcc'] = 'ripatel272@gmail.com, anakha.1@gmail.com'
msg = MIMEText(html_content,'html')
message.attach(msg)
# Initialize the Drive API

for file_attach in files_2_attach:
    with open(file_attach, 'rb') as f:
        mime_base = MIMEBase('application','octet-stream')
        mime_base.set_payload(f.read())
        encoders.encode_base64(mime_base)
        mime_base.add_header('Content-Disposition','attachment',filename=file_attach)
        message.attach(mime_base)

encoded_message = base64.urlsafe_b64encode(message.as_bytes()).decode()
create_message = {'raw':encoded_message}

send_message = service.users().messages().send(userId="me", body=create_message).execute()


In [48]:
service

## Get contacts

In [80]:
people_service

In [72]:
gmail_emails = ag.get_contacts(people_service)
','.join(gmail_emails)

'priabraham@gmail.com,lizziehinson96@gmail.com,ilija.medan@vanderbilt.edu,heathergc2k19@gmail.com,Cthomp28@gmail.com,selliott@fwpublishing.com,michael.z.reynolds@vanderbilt.edu,lauren.n.kasper@vanderbilt.edu,riyengar828@gmail.com,jennifer.l.james@vanderbilt.edu,alec.myszka@me.com,info@chefpaulette.net,roman.berens@vanderbilt.edu,beardeadsaint22@outlook.com,Kyle.Hawkins@vanderbilt.edu,a.rockafellar@vanderbilt.edu,margiedz@yahoo.com,Ashley.e.brammer@vanderbilt.edu,Ccoca@fisk.edu,Patricia.a.ward.3@vanderbilt.edu,Kelly.gravity@gmail.com,donald.pickert@VANDERBILT.EDU,coolspringsmoderndentistry@smilegeneration.com,mmarcano@fisk.edu,william.k.teets@vanderbilt.edu,jonathan.bird@vanderbilt.edu,alexandra.f.rockafellar@vanderbilt.edu,info@bsasnashville.com,thryan13@gmail.com,mattdoc@hotmail.com,pencekt@gmail.com,edgar.carrillo@vanderbilt.edu,info@bsasnashville.com,christina.mcgahan@vanderbilt.edu,mary.e.kaldor@vanderbilt.edu,vaiby@hotmail.com,nicholas-tyler.howard@vanderbilt.edu,grimblw@vanderbil

## Get contacts from google sheets

### Get sheet id names and ID

In [42]:
service_drive = build('drive', 'v3', credentials=creds)

# Define the query to filter only Google Sheets files
query = "mimeType='application/vnd.google-apps.spreadsheet'"

# Call the Drive API to list files
results = service_drive.files().list(q=query, fields="files(id, name)").execute()
items = results.get('files', [])
sheets_part_addy = 'https://docs.google.com/spreadsheets/d/'
sheets_info = {}
if not items:
    print('No files found.')
else:
    for item in items:
        sheets_info[item['name']] = {'url':f"{sheets_part_addy}{item['id']}/edit",
                                     'id':item['id']}
        

In [43]:
sheets_info

{'Speaker List/Schedule/stats': {'url': 'https://docs.google.com/spreadsheets/d/19AW_a_DHoy4z3WJOqXvFceI_fP6crSyp0afw5wdWdds/edit',
  'id': '19AW_a_DHoy4z3WJOqXvFceI_fP6crSyp0afw5wdWdds'},
 'Astronomy on Tap - Email Sign-up (Responses)': {'url': 'https://docs.google.com/spreadsheets/d/1qUAXI_XPnOSKaooCKvS666iLeKi77JiRs3IzpdKTttg/edit',
  'id': '1qUAXI_XPnOSKaooCKvS666iLeKi77JiRs3IzpdKTttg'},
 'AoT Satellites': {'url': 'https://docs.google.com/spreadsheets/d/1P3rM9BU30R_SiaK_3315jEYsdTWL9XtD8NV67HZlIeM/edit',
  'id': '1P3rM9BU30R_SiaK_3315jEYsdTWL9XtD8NV67HZlIeM'},
 'Speaker Information  (Responses)': {'url': 'https://docs.google.com/spreadsheets/d/1gFugAoIBDqFBq5ej5ZwECh3vdqzpMikQBRStoahFL6o/edit',
  'id': '1gFugAoIBDqFBq5ej5ZwECh3vdqzpMikQBRStoahFL6o'},
 'AoT Games': {'url': 'https://docs.google.com/spreadsheets/d/1exY5yxZx702anzm0_9R-5Bd-JADTZXUKiXTctU3Q4Ns/edit',
  'id': '1exY5yxZx702anzm0_9R-5Bd-JADTZXUKiXTctU3Q4Ns'},
 'AoT Nashville Speaker Solicitation (Responses)': {'url': 'http

In [40]:
import re
sheets_2_find = '.*Email*.*Responses*.'
regex = re.compile(sheets_2_find, re.IGNORECASE)
try:
    name_of_email_sheet = list(filter(regex.match,sheets_info.keys()))[0]
except:
    print(f'Could not find sheet with regex: {sheets_2_find}')
    print('Reason unknown')

In [41]:
name_of_email_sheet

'Astronomy on Tap - Email Sign-up (Responses)'

## Get sheet data

In [50]:
import pandas as pd

In [47]:
spreadsheet_id = sheets_info[name_of_email_sheet]['id']

In [67]:
values[0]

['Timestamp', 'Name', 'Email Address', 'Email Address2']

In [90]:
service_sheets = build('sheets','v4',credentials=creds)
sheet = service_sheets.spreadsheets()

sheet_range = '!A:Z'
result = sheet.values().get(spreadsheetId=spreadsheet_id,range=sheet_range).execute()
values = result.get('values',[])
if not values:
    print('no data found')

df = pd.DataFrame(values[1:],columns=values[0][0:-1])
sheet_emails = df['Email Address']
sheet_emails

,Email Address
0,mmarcano@fisk.edu
1,edgar.carrillo@vanderbilt.edu
2,ryan.nowicki@vanderbilt.edu
3,mary.e.kaldor@vanderbilt.edu
4,Patricia.a.ward.3@vanderbilt.edu
...,...
70,Kyle.Hawkins@vanderbilt.edu
71,jordandias617@gmail.com
72,tjpencarinha@gmail.com
73,Joey Grassi


In [92]:
# Initial DataFrame with a single column
data = {
    'Email Address': [
        'mmarcano@fisk.edu', 'edgar.carrillo@vanderbilt.edu', 'ryan.nowicki@vanderbilt.edu',
        'mary.e.kaldor@vanderbilt.edu', 'Patricia.a.ward.3@vanderbilt.edu', 'Kyle.Hawkins@vanderbilt.edu',
        'jordanias617@gmail.com', 'tjpencarinha@gmail.com', 'Joey Grassi', 'mbsnahsville@outlook.com'
    ]
}
df = pd.DataFrame(data)

# Display initial DataFrame
print("Initial DataFrame:")
print(df)

# List of new email addresses to add
new_emails = ['new.email1@example.com', 'new.email2@example.com', 'new.email3@example.com']

# Convert the new email addresses list to a DataFrame
new_df = pd.DataFrame({'Email Address': new_emails})

# Concatenate the new DataFrame with the existing DataFrame
df = pd.concat([df, new_df], ignore_index=True)

# Display updated DataFrame
print("\nUpdated DataFrame:")
print(df)

Initial DataFrame:
                      Email Address
0                 mmarcano@fisk.edu
1     edgar.carrillo@vanderbilt.edu
2       ryan.nowicki@vanderbilt.edu
3      mary.e.kaldor@vanderbilt.edu
4  Patricia.a.ward.3@vanderbilt.edu
5       Kyle.Hawkins@vanderbilt.edu
6            jordanias617@gmail.com
7            tjpencarinha@gmail.com
8                       Joey Grassi
9          mbsnahsville@outlook.com

Updated DataFrame:
                       Email Address
0                  mmarcano@fisk.edu
1      edgar.carrillo@vanderbilt.edu
2        ryan.nowicki@vanderbilt.edu
3       mary.e.kaldor@vanderbilt.edu
4   Patricia.a.ward.3@vanderbilt.edu
5        Kyle.Hawkins@vanderbilt.edu
6             jordanias617@gmail.com
7             tjpencarinha@gmail.com
8                        Joey Grassi
9           mbsnahsville@outlook.com
10            new.email1@example.com
11            new.email2@example.com
12            new.email3@example.com


### Merge both lists

In [117]:
gmail_df = pd.DataFrame({'Email Address':gmail_emails})
new_email_df = pd.concat([gmail_df, sheet_emails],ignore_index=True)

In [118]:
new_email_df.drop_duplicates(inplace=True)

In [122]:
import numpy

In [119]:
new_email_df

,Email Address
0,priabraham@gmail.com
1,lizziehinson96@gmail.com
2,ilija.medan@vanderbilt.edu
3,heathergc2k19@gmail.com
4,Cthomp28@gmail.com
...,...
115,savion.j.johnson@vanderbilt.edu
121,tmwellington2@gmail.com
123,ccoca@fisk.edu
133,lauren.h.kasper@vanderbilt.edu


In [125]:
np.concatenate(new_email_df.to_numpy())

array(['priabraham@gmail.com', 'lizziehinson96@gmail.com',
       'ilija.medan@vanderbilt.edu', 'heathergc2k19@gmail.com',
       'Cthomp28@gmail.com', 'selliott@fwpublishing.com',
       'michael.z.reynolds@vanderbilt.edu',
       'lauren.n.kasper@vanderbilt.edu', 'riyengar828@gmail.com',
       'jennifer.l.james@vanderbilt.edu', 'alec.myszka@me.com',
       'info@chefpaulette.net', 'roman.berens@vanderbilt.edu',
       'beardeadsaint22@outlook.com', 'Kyle.Hawkins@vanderbilt.edu',
       'a.rockafellar@vanderbilt.edu', 'margiedz@yahoo.com',
       'Ashley.e.brammer@vanderbilt.edu', 'Ccoca@fisk.edu',
       'Patricia.a.ward.3@vanderbilt.edu', 'Kelly.gravity@gmail.com',
       'donald.pickert@VANDERBILT.EDU',
       'coolspringsmoderndentistry@smilegeneration.com',
       'mmarcano@fisk.edu', 'william.k.teets@vanderbilt.edu',
       'jonathan.bird@vanderbilt.edu',
       'alexandra.f.rockafellar@vanderbilt.edu', 'info@bsasnashville.com',
       'thryan13@gmail.com', 'mattdoc@hotmail.com

In [107]:
type(new_email_df)

pandas.core.frame.DataFrame

In [78]:
all_emails = list(sheet_emails.values) + gmail_emails
len(sheet_emails), len(gmail_emails), len(sheet_emails) + len(gmail_emails), len(all_emails)

(75, 75, 150, 150)

In [79]:
email_string = ','.join(all_emails)
email_string

'mmarcano@fisk.edu,edgar.carrillo@vanderbilt.edu ,ryan.nowicki@vanderbilt.edu,mary.e.kaldor@vanderbilt.edu,Patricia.a.ward.3@vanderbilt.edu,Ashley.e.brammer@vanderbilt.edu,ilija.medan@vanderbilt.edu,kaitlyn.t.lane@vanderbilt.edu,Cthomp28@gmail.com,dpe00a@gmail.com,Edgarc.ec@gmail.com,a.rockafellar@vanderbilt.edu,info@chefpaulette.net,diyac.03@gmail.com,sade_johnson51@yahoo.com,sade.johnson@nashville.gov,Brian.Thomas.Garton@gmail.com,allie.j.whalen@gmail.com,jrportwood65@gmail.com,spacepiratecaptainjack@gmail.com,paul.hibbeler@gmail.com,m.shane.hutson@gmail.com,Giannamv@gmail.com,warrickelaine@gmail.com,grimblw@vanderbilt.edu,Kelly.gravity@gmail.com,carolyn.drake@vanderbilt.edu,christina.mcgahan@vanderbilt.edu,erika.taylor@vanderbilt.edu,sfrederi7@gmail.com,reevesaf@gmail.com,mcdigman@gmail.com,kHammer8280@aol.com,cate.enstrom@gmail.com,pencekt@gmail.com,drew@newtonscannon.com,thryan13@gmail.com,dyerobservatory@vanderbilt.edu,donald.pickert@vanderbilt.edu,william.k.teets@vanderbilt.edu,

# EventBrite links

In [131]:
import requests
import json

In [134]:
API_EVENTBRITE_TOKEN = None

# Eventbrite API endpoint to get user's events
EVENTS_URL = 'https://www.eventbriteapi.com/v3/users/me/events/'

# Function to get all events
def get_eventbrite_events(api_token):
    headers = {
        'Authorization': f'Bearer {api_token}',
    }

    response = requests.get(EVENTS_URL, headers=headers)
    
    if response.status_code != 200:
        raise Exception(f"Error fetching events: {response.status_code}")

    events_data = response.json()
    events = events_data.get('events', [])

    return events

# Function to extract event names and hyperlinks
def extract_event_details(events):
    event_details = []
    
    for event in events:
        name = event['name']['text']
        url = event['url']
        event_details.append({'name': name, 'url': url})
    
    return event_details


In [135]:
events = get_eventbrite_events(API_EVENTBRITE_TOKEN)
event_details = extract_event_details(events)

Exception: Error fetching events: 404